# Study simple gradient policy training

In [ ]:
import tensorflow as tf
import numpy as np
import gym
from gym.spaces import Discrete, Box

from IPython.core.debugger import set_trace


In [ ]:
def mlp(x, sizes, activation=tf.tanh, output_activation=None):
    # Build a feedforward neural network.
    for size in sizes[:-1]:
        x = tf.layers.dense(x, units=size, activation=activation)
    return tf.layers.dense(x, units=sizes[-1], activation=output_activation)

def train(env_name='CartPole-v0', hidden_sizes=[32], lr=1e-2, 
          epochs=5, batch_size=5000, render=False):

    # make environment, check spaces, get obs / act dims
    env = gym.make(env_name)
    assert isinstance(env.observation_space, Box), \
        "This example only works for envs with continuous state spaces."
    assert isinstance(env.action_space, Discrete), \
        "This example only works for envs with discrete action spaces."

    obs_dim = env.observation_space.shape[0]
    n_acts = env.action_space.n

    # make core of policy network
    obs_ph = tf.placeholder(shape=(None, obs_dim), dtype=tf.float32)
    logits = mlp(obs_ph, sizes=hidden_sizes+[n_acts])
    print(f'logits shape: {logits.shape}')
    
    tmp = tf.multinomial(logits=logits,num_samples=1)
    print(f'multinomial shape: {tmp.shape}')
    
    # make action selection op (outputs int actions, sampled from policy)
    actions = tf.squeeze(tmp, axis=1)
    print(f'actons shape: {actions.shape}')

    # make loss function whose gradient, for the right data, is policy gradient
    weights_ph = tf.placeholder(shape=(None,), dtype=tf.float32)
    act_ph = tf.placeholder(shape=(None,), dtype=tf.int32)
    action_masks = tf.one_hot(act_ph, n_acts)
    log_probs = tf.reduce_sum(action_masks * tf.nn.log_softmax(logits), axis=1)
    loss = -tf.reduce_mean(weights_ph * log_probs)

    # make train op
    train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

    # for training policy
    def train_one_epoch():
        # make some empty lists for logging.
        batch_obs = []          # for observations
        batch_acts = []         # for actions
        batch_weights = []      # for R(tau) weighting in policy gradient
        batch_rets = []         # for measuring episode returns
        batch_lens = []         # for measuring episode lengths

        # reset episode-specific variables
        obs = env.reset()       # first obs comes from starting distribution
        done = False            # signal from environment that episode is over
        ep_rews = []            # list for rewards accrued throughout ep

        # render first episode of each epoch
        finished_rendering_this_epoch = False

        # collect experience by acting in the environment with current policy
        while True:

            # rendering
            if (not finished_rendering_this_epoch) and render:
                env.render()

            # save obs
            batch_obs.append(obs.copy())

            # act in the environment
            act = sess.run(actions, {obs_ph: obs.reshape(1,-1)})[0]
            obs, rew, done, _ = env.step(act)

            # save action, reward
            batch_acts.append(act)
            ep_rews.append(rew)

            if done:
                # if episode is over, record info about episode
                ep_ret, ep_len = sum(ep_rews), len(ep_rews)
                batch_rets.append(ep_ret)
                batch_lens.append(ep_len)

                # the weight for each logprob(a|s) is R(tau)
                batch_weights += [ep_ret] * ep_len

                # reset episode-specific variables
                obs, done, ep_rews = env.reset(), False, []

                # won't render again this epoch
                finished_rendering_this_epoch = True

                # end experience loop if we have enough of it
                if len(batch_obs) > batch_size:
                    break

        # take a single policy gradient update step
        batch_loss, _ = sess.run([loss, train_op],
                                 feed_dict={
                                    obs_ph: np.array(batch_obs),
                                    act_ph: np.array(batch_acts),
                                    weights_ph: np.array(batch_weights)
                                 })
        return batch_loss, batch_rets, batch_lens

    # training loop
    for i in range(epochs):
        batch_loss, batch_rets, batch_lens = train_one_epoch()
        print('epoch: %3d \t loss: %.3f \t return: %.3f \t ep_len: %.3f'%
                (i, batch_loss, np.mean(batch_rets), np.mean(batch_lens)))

In [ ]:
train(env_name='CartPole-v0', render=False, lr=1e-2)

In [ ]:
isess = tf.InteractiveSession()

In [ ]:
in_probab = np.array([[1/9.,1/4.,7/3.]])
print(in_probab.shape)
samples = tf.multinomial(logits=tf.log(in_probab), num_samples=1)
tmp = isess.run(tf.squeeze(samples, axis=1))
print(tmp.shape)

In [ ]:
count = [0, 0, 0]
for _ in range(1000):
    pick = isess.run(tf.squeeze(samples, axis=1))[0]
    count[pick] += 1

print(count)

In [ ]:
isess.close()

In [ ]:
size = [32]+[2]

In [ ]:
print(size[:-1])

In [ ]:
print(size[-1])

In [ ]:
isess = tf.InteractiveSession()
in_probab = np.array([[0.1, 0.2, 0.7]])
print(in_probab.shape)
samples = tf.multinomial(logits=tf.log(in_probab), num_samples=10000)
tmp = isess.run(tf.squeeze(samples))
isess.close()

print(tmp.shape)
print(tmp)

from collections import Counter
Counter(tmp)

In [ ]:
# work through exercise 2 solutions

import tensorflow as tf
import numpy as np

In [ ]:
x = tf.constant(np.arange(5))
y = tf.constant(np.arange(5).reshape(-1,1))

In [ ]:
sess = tf.InteractiveSession()
sess.run(x)
sess.run(y)

In [ ]:
sess.close()

In [ ]:
x = np.arange(5)

In [ ]:
print(x.shape)

In [ ]:
print(x.reshape(-1,1).shape)

In [ ]:
env = gym.make("LunarLander-v2")

In [ ]:
env.render(mode='rgb_array')

In [ ]:
env.action_space.sample

In [1]:
import pybullet_envs
import gym
import spinup

/home/user/.local/lib/python3.6/site-packages/gym/envs/registration.py:64: UserWarning: register(timestep_limit=1000) is deprecated. Use register(max_episode_steps=1000) instead.
  warnings.warn("register(timestep_limit={}) is deprecated. Use register(max_episode_steps={}) instead.".format(timestep_limit, timestep_limit))


In [2]:
import baselines

ModuleNotFoundError: No module named 'baselines'

In [ ]:
env = gym.make('InvertedDoublePendulumBulletEnv-v0')

In [ ]:
pybullet.connect(pybullet.DIRECT)

In [ ]:
env.reset()

In [ ]:
import spinup

In [ ]:
print(env)

In [ ]:
spinup.ddpg(env)

In [ ]:
import pybullet_envs

In [ ]:
env = gym.make("HalfCheetahBulletEnv-v0")

In [ ]:
env.render(mode="human")